### Northwind Traders Analytics

- Connect to PostgreSQL database and load SQL extension
- Perform data exploration to view all base tables
- Create tables with common joins for quering throughout the rest of the project

In [4]:
%load_ext sql
%sql postgresql://postgres@localhost:5432/northwind

In [5]:
%%sql

SELECT table_name, table_type 
  FROM information_schema.tables 
 WHERE table_schema = 'public';

 * postgresql://postgres@localhost:5432/northwind
17 rows affected.


table_name,table_type
territories,BASE TABLE
order_details,BASE TABLE
employee_territories,BASE TABLE
us_states,BASE TABLE
customers,BASE TABLE
orders,BASE TABLE
employees,BASE TABLE
shippers,BASE TABLE
products,BASE TABLE
categories,BASE TABLE


In [119]:
%%sql 

CREATE OR REPLACE VIEW customer_orders AS 
SELECT c.*, o.order_id, o.employee_id, o.order_date, o.required_date, o.shipped_date, o.ship_via, o.freight, o.ship_name, o.ship_address, o.ship_city, o.ship_region, o.ship_postal_code, o.ship_country
  FROM customers AS c
  JOIN orders AS o 
    ON c.customer_id = o.customer_id;

CREATE OR REPLACE VIEW product_quantity_orders AS
SELECT p.*, od.quantity, od.discount, o.*
  FROM products AS p
  JOIN order_details AS od ON p.product_id = od.product_id
  JOIN orders AS o ON od.order_id = o.order_id
 ORDER BY p.product_id;

CREATE OR REPLACE VIEW employee_orders AS 
SELECT e.last_name, e.first_name, e.title, e.title_of_courtesy, e.birth_date, e.hire_date, address, city, region, postal_code, country, home_phone, extension, notes, reports_to, photo_path, o.*
  FROM employees AS e
  JOIN orders AS o ON e.employee_id = o.employee_id
 ORDER BY o.employee_id;

 * postgresql://postgres@localhost:5432/northwind
Done.
Done.
Done.


[]

#### Employees Sales Performance

- Rank employees based on their total sales amount

In [21]:
%%sql

WITH aggregate_sales AS(
SELECT e.employee_id, SUM(od.unit_price * od.quantity * (1 - od.discount)) AS total_sales
  FROM employees AS e
  JOIN orders AS o ON e.employee_id = o.employee_id
  JOIN order_details AS od ON o.order_id = od.order_id
 GROUP BY e.employee_id)

SELECT employee_id, ROUND(total_sales::numeric,2) AS total_sales, RANK() OVER(ORDER BY total_sales DESC) AS employee_rank
  FROM aggregate_sales;

 * postgresql://postgres@localhost:5432/northwind
9 rows affected.


employee_id,total_sales,employee_rank
4,232890.85,1
3,202812.84,2
1,192107.60,3
2,166537.76,4
8,126862.28,5
7,124568.23,6
9,77308.07,7
6,73913.13,8
5,68792.28,9


- Calculated the total sales for each employee by multiplying unit price * quantity and subtracting any discounts at the order level
- Utilized ranking window function to identify high and low performers
- High performers are employees 4,3,1 and low performers are employees 9,6,5

#### Monthly Sales Trends

- Visualize the progress of the sales and identify trends that might shape the company's future strategies
- Aggregate sales data at a monthly level and calculating a running total of sales by month
- Analyze the month-over-month sales growth rate

In [117]:
%%sql

WITH monthly_sales AS (
SELECT DATE_TRUNC('month', order_date) AS order_month, SUM(unit_price * quantity * (1 - discount)) AS monthly_sales
  FROM orders 
  JOIN order_Details ON orders.order_id = order_details.order_id
 GROUP BY order_month
 ORDER BY order_month ASC)

SELECT order_month, ROUND(monthly_sales::numeric,2) AS monthly_sales, 
       ROUND(LAG(monthly_sales) OVER w::numeric,2) AS previous_month_sales,
       ROUND((monthly_sales::numeric -  LAG(monthly_sales) OVER w)::numeric / LAG(monthly_sales) OVER w::numeric * 100,2) AS MoM_growth_rate,
       ROUND(SUM(monthly_sales) OVER w::numeric,2) AS running_total
  FROM monthly_sales
WINDOW w AS (ORDER BY order_month);

 * postgresql://postgres@localhost:5432/northwind
23 rows affected.


order_month,monthly_sales,previous_month_sales,mom_growth_rate,running_total
1996-07-01 00:00:00-04:00,27861.90,None,None,27861.90
1996-08-01 00:00:00-04:00,25485.28,27861.90,-8.53,53347.17
1996-09-01 00:00:00-04:00,26381.40,25485.28,3.52,79728.57
1996-10-01 00:00:00-04:00,37515.72,26381.40,42.21,117244.30
1996-11-01 00:00:00-05:00,45600.05,37515.72,21.55,162844.34
1996-12-01 00:00:00-05:00,45239.63,45600.05,-0.79,208083.97
1997-01-01 00:00:00-05:00,61258.07,45239.63,35.41,269342.04
1997-02-01 00:00:00-05:00,38483.63,61258.07,-37.18,307825.68
1997-03-01 00:00:00-05:00,38547.22,38483.63,0.17,346372.90
1997-04-01 00:00:00-05:00,53032.95,38547.22,37.58,399405.85


#### High Value Customers
- Identify customers with above-average order values
- Rank customers based on the quantity of above-average orders and highlight the top 5 customers
- Identify the top 20% of customers by total purchase volume.

In [63]:
%%sql

WITH customer_order_totals AS (
SELECT c.customer_id, c.order_id, SUM(o.unit_price * o.quantity * (1 - o.discount)) AS order_total
  FROM customer_orders AS c
  JOIN order_details AS o ON c.order_id = o.order_id
 GROUP BY c.customer_id, c.order_id
 ORDER BY c.customer_id, c.order_id),

order_categorization AS (
SELECT customer_id, order_id, order_total,
       CASE
        WHEN order_total > AVG(order_total) OVER w THEN 'Above Average'
        WHEN order_total <= AVG(order_total) OVER w THEN 'Average/Below Average'
       END AS order_category
  FROM customer_order_totals
WINDOW w AS (ORDER BY order_total RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)),

customer_ranking AS (
SELECT customer_id, COUNT(DISTINCT order_id) AS above_avg_order_count,
       ROW_NUMBER() OVER(ORDER BY COUNT(DISTINCT order_id) DESC) AS customer_rnk
  FROM order_categorization
 WHERE order_category = 'Above Average'
 GROUP BY customer_id)

SELECT customer_id, above_avg_order_count
  FROM customer_ranking
 WHERE customer_rnk <=5;

 * postgresql://postgres@localhost:5432/northwind
5 rows affected.


customer_id,above_avg_order_count
ERNSH,26
SAVEA,26
QUICK,22
HUNGO,11
RATTC,10


The top 5 customers with the most above-average orders are ERNSH, SAVEA, QUICK, HUNGO & RATTC.

#### Product and Category Performance
- Assess the sales composition across different product categories by finding the percent of total sales for each

In [123]:
%%sql

WITH aggregate_sales AS (
SELECT p.category_id, c.category_name, ROUND(SUM(unit_price * quantity * (1 - discount))::numeric,2) AS total_sales
  FROM product_quantity_orders AS p
  JOIN categories AS c ON p.category_id = c.category_id
 GROUP BY p.category_id, c.category_name
 ORDER BY total_sales DESC)

SELECT category_id, category_name, total_sales, ROUND(total_sales / (SELECT SUM(total_sales) FROM aggregate_sales) * 100.0,2) AS perc_of_total,
       SUM(total_sales) OVER(ORDER BY total_sales DESC) AS running_total,
       SUM(ROUND(total_sales / (SELECT SUM(total_sales) FROM aggregate_sales) * 100.0,2)) OVER(ORDER BY total_sales DESC) AS running_percent
  FROM aggregate_sales
 ORDER BY total_sales DESC;

 * postgresql://postgres@localhost:5432/northwind
8 rows affected.


category_id,category_name,total_sales,perc_of_total,running_total,running_percent
1,Beverages,288694.65,21.33,288694.65,21.33
4,Dairy Products,251147.60,18.56,539842.25,39.89
3,Confections,179863.65,13.29,719705.90,53.18
6,Meat/Poultry,174622.55,12.90,894328.45,66.08
8,Seafood,137989.31,10.20,1032317.76,76.28
2,Condiments,113692.19,8.40,1146009.95,84.68
7,Produce,105745.70,7.81,1251755.65,92.49
5,Grains/Cereals,101646.94,7.51,1353402.59,100.00


The most popular product categories are beverages, dairy products and confections which make up 53.18% of all sales ($719,705.90)

#### Top Products Per Category
- Drill down further into each group to determine the top three items sold in each product category

In [93]:
%%sql

WITH sales_by_product AS (
SELECT c.category_name, p.product_name, ROUND(SUM(unit_price * quantity * (1 - discount))::numeric,2) AS total_sales,
       ROW_NUMBER() OVER(PARTITION BY c.category_name ORDER BY SUM(unit_price * quantity * (1 - discount))::numeric DESC) AS product_ranking
  FROM product_quantity_orders AS p
  JOIN categories AS c ON p.category_id = c.category_id
 GROUP BY 1,2)

SELECT category_name, product_name, total_sales
  FROM sales_by_product
 WHERE product_ranking <= 3

 * postgresql://postgres@localhost:5432/northwind
24 rows affected.


category_name,product_name,total_sales
Beverages,Côte de Blaye,153897.17
Beverages,Ipoh Coffee,25109.10
Beverages,Chang,17719.40
Condiments,Vegie-spread,18343.62
Condiments,Sirop d'érable,15022.35
Condiments,Louisiana Fiery Hot Pepper Sauce,14893.93
Confections,Tarte au sucre,50737.09
Confections,Sir Rodney's Marmalade,24199.56
Confections,Gumbär Gummibärchen,21662.69
Dairy Products,Raclette Courdavault,76683.75


#### Additional Metrics
- Compare each employee's sales performance to the average sales performance of all employees to evaluate their contributions.
- Analyze the cumulative sales for each product category over a specified period to understand each category's growth.
- Identify products that perform exceptionally well in terms of sales. Specifically, products whose total sales exceed the 75th percentile.